# eBay Assistant v2 | Development Notebook

In [40]:
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
load_dotenv()
import openai
from IPython.display import Markdown, display, HTML, Image

os.chdir(os.path.dirname(os.getcwd()))

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import unquote
from readabilipy import simple_json_from_html_string
from langchain.schema import Document

In [3]:
from src.doc_store.ebay_scraper import eBayWebSearch, AverageSalePrice, eBayProduct

* Test "query" based on an item description from the existing app

In [4]:
query = 'Vintage Appleton Industrial Vented Light Fixture'

* The `AverageSalePrice` function searches the eBay API for sold items matching a description
* It excludes extreme outliers, then calculates average selling price (broken down by item cost & shipping) across all results
* Filters can be adjusted as needed, and the actual items can be brought in as well

In [5]:
averagePrice = AverageSalePrice(query=query, country='us', condition='all')
print(averagePrice)

Item Description: Vintage Appleton Industrial Vented Light Fixture
Average Price (based on 53 sold items):
* Item: $48.78 ($2.886 to $175.0)
* Shipping: $15.37 ($0.0 to $45.12)
* Total: $64.15


* The `eBayWebSearch` function works the same as searching eBay via a browser
* Given a query it returns a list of `eBayProduct` objects shown below
* Results can be formatted and easily added to an OpenAI prompt
* **Note:** Search results **contain links** to the primary image for the listing - GPT-4 with vision can **read images from links!**

In [6]:
search_res = eBayWebSearch(query)
len(search_res)

73

* Example of the `eBayProduct` search result

In [54]:
search_res[1].model_dump()

{'ebay_id': 386723431936,
 'item': {'title': 'VTG Appleton A-51 Vented Explosion-Proof Industrial Light - Man Cave Garage Bar',
  'link': Url('https://www.ebay.com/itm/386723431936?itmmeta=01HRNWCDJAYKKGCP4HAYYJKF4P&hash=item5a0a834600:g:9QwAAOSwpDlluDud&itmprp=enc%3AAQAIAAABAGP2goulIxqfc2L%2BgWJXqhnBKYLugKAE%2FnzT6kcmqO0xyVMT8z5I3GrddMrK8C%2BEzcvwJcmoVxsVbajkeBGKGld6e2GLR09dPL9NMa7xBu5zK2Qhi%2BxdsGiKmgI%2FzaBXaSet%2Bn7cXdE6jhUrShpzEmqOW8%2Bjds6RzMLXv%2BtBawrPlerXDmLL%2F4sZAJ4tjDp2O%2Fu%2FyDQd77p5dcanqTns5RmRuh9LDAA2QeFB1%2B8ApyqsqHw4tVPVNkL5qw06JOJn3MbHOpoSN2YCqzbuybV8F6IDUSDImDXe%2BKClHTULnVJagur%2BzcAs6JFQ6%2BpBylk5kQca6ZqBDWNIIclnceSKuXE%3D%7Ctkp%3ABk9SR6LZsbzFYw'),
  'price': 39.99,
  'image_url': Url('https://i.ebayimg.com/thumbs/images/g/9QwAAOSwpDlluDud/s-l300.jpg'),
  'description': 'eBay\nTransform your space with this vintage Appleton Electric A-51 Vented Explosion-Proof Industrial Light Fixture - a perfect addition to your man cave, industrial garage, or home bar decor!\nCr

* Example of formatted search results

In [11]:
for i in range(len(search_res[:4])):
    print(search_res[i])

Vintage Appleton Explosion Proof Vented Industrial Ceiling Light Lamp Fixture
Pre-Owned
$39.99
+$35.10 shipping

Item description from the seller:
Vintage Appleton Explosion Proof Vented Industrial Ceiling Light Lamp Fixture
Parts Only - Light is untested
200W - 250V - Type EVA - Catalog AA 99
Measures
8" in dia x
16
" in height (not including threaded rod)
all approx
Fresh Barn Find the Aluminum housing has oxidation
Glass Looks Good - Has a
Porcelain
socket
Down Rod will be removed for shipping
Please study all the photos
Feel Free to ask any questions
Check Back I found more unusual Light Fixtures in this Barn Including a few Holophane shades et

eBay item number: 155896783352
Listing URL: https://www.ebay.com/itm/155896783352?itmmeta=01HRNWCDJAHJ1Q0GXC01V3JV30&hash=item244c2c21f8:g:PrUAAOSwK2xlR49E&itmprp=enc%3AAQAIAAAA4B8%2B2Q1W6ISeGsSM37CcksptgXDgAzaL74Hvnh0stjTGog%2FWSESj7YNo%2Bci6OxmTcKQkmhHwCAunkkIhB1aso0yxwO9RSKxdafVrFg6sCFy3fiEvDwz4LckinD%2Fj7VbxuGNdSo0BOUaNkDOCmIRuWVSm4ULld

* Example showing `eBayProduct` sorted by price (ascending or descending)

* Least expensive

In [14]:
sorted_results = eBayProduct.sort_by_price(search_res, reverse=False)
for i in range(len(sorted_results[:4])):
    print(sorted_results[i])

Vintage Appleton Light Fixture Clear Ribbed Glass Globe & Cage Unknown Model
Pre-Owned
$19.95
+$13.05 shipping

Item description from the seller:
Appleton Products light globe and cage. Have some marks from use as shown in pictures.  The inside of the globe is ribbed.  the opening is 5.25" across and the globe is 6" tall.  No model numbers on it just Appleton logo at bottom of globe. It is preowned.
LOC 4

eBay item number: 123718689975
Listing URL: https://www.ebay.com/itm/123718689975?itmmeta=01HRNWCDJAKGTS3HVPWYXDQTGE&hash=item1cce3564b7:g:lZUAAOSwGUBaRVLx&itmprp=enc%3AAQAIAAABAMA9Cd93bTMPT6uSQNJZIvQZpqFSF6pup65XR1TEzQ3LUEqfsEdQj0jW%2FBDIRm%2F7jdb6Ejzx1ir9ESDeSjSudsSH4zee7JMTeu7rI4CCLpaqXzCEORphQ7%2B0KjYKVFwbAYL8voJsabpYg0EM3HY%2FGtfSbmJJg3nK1RjHN5AR14j%2FEO74xfikoc7kIfyBuTzkwlpntJ1%2F37MIkl3f7G9hSY5B%2BP%2BSx63GGnZpRC659yXvj6%2BPZzkUEsIoIl%2BgtUDgdPoQjw8VUao9qSghF63hS3TZ7tWc87fviwNOl5LmVDV6eIKLW1CLnGFYxkEd5fe%2BiBui1ePtpGSN81C9v8LilAQ%3D%7Ctkp%3ABFBMpNmxvMVj
Image URL: https://i.eb

* Most expensive

In [9]:
sorted_high_results = eBayProduct.sort_by_price(search_res, reverse=True)
for i in range(len(sorted_high_results[:4])):
    print(sorted_high_results[i])

Set of 4 Appleton 12" Porcelain Flush Mount Industrial Green Enamel Barn Light
Pre-Owned
$660.0
+$25.00 shipping

Item description from the seller:
SOLD AS A SET OF FOUR
The absolute perfect light for lower ceilings. Excellent for use in hallways, small rooms or almost any room with four ceiling boxes. Garages are also a good spot for these. Lot's of garages need exactly four of these. Also, the overhead garage door will miss these!
Nice little lights, perfect for 8 foot or lower ceiling as they only need 6 inches of space.
These heavy porcelain covered steel light fixtures were manufactured by Appleton,  Circa 1936 patent.
Push and turn to lower the shade for cleaning.  A clean shade shines brighter.
All I had to do was clean the shades, disassemble, re-wire and reassemble.
The shades are 12 inches in diameter. The total height is only  inches.
In order to safely ship the lights, they will be disassembled, to be easily reassembled by the buyer.
Please feel free to ask questions prior 

## Image comparison test

* Function to send multiple images to GPT-4

In [ ]:
from openai import OpenAI

client = OpenAI()

def compare_item_images(base_img_url: str, comp_image_url: str):
    response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": "I'm planning to list the item in the first image on eBay and looking for good 'comps' to help determine the price. Please review and asses whether the second image is similar enough to use. Include a highly concise explanation as to why.",
            },
            {
            "type": "image_url",
            "image_url": {
                "url": base_img_url,
            },
            },
            {
            "type": "image_url",
            "image_url": {
                "url": comp_image_url,
            },
            },
        ],
        }
    ],
    max_tokens=300,
    )
    return response.choices[0]

* Target image to compare against

In [42]:
target_image = "https://i.ebayimg.com/thumbs/images/g/9QwAAOSwpDlluDud/s-l300.jpg"
display(Image(url=target_image))

* First a bad comp we would not want to use

In [41]:
bad_comp_image = str(sorted_results[11].item.image_url)
display(Image(url=bad_comp_image))

In [27]:
# Testing a bad match
image_comp = compare_item_images(target_image, bad_comp_image)
Markdown(image_comp.message.content)

The first image depicts an industrial or vintage light fixture with a distinctive red label and an attached wiring conduit. The second image shows a different light fixture that lacks the attached wiring conduit and has a different protective cage design.

The items are not similar enough to use as comps for pricing because:
- The designs are notably different, indicating they may serve different functions or fit different aesthetics.
- The presence of the wiring conduit and the red label in the first item adds to its characteristics, possibly affecting its value.
- The condition and age of the items might differ, which can significantly influence the price.

Use comps that more closely match the design, features, and condition of your item for accurate pricing.

* This time a seemingly decent comp item

In [43]:
good_comp_image = str(sorted_results[7].item.image_url)
display(Image(url=good_comp_image))

In [35]:
# Looks like a good match to me
image_comp = compare_item_images(target_image, good_comp_image)
Markdown(image_comp.message.content)

Yes, the second image is similar enough to use for price comparison. Both images show vintage industrial light fixtures with caged, glass globes and similar design aesthetics. They appear to be of a similar era and style, which is pertinent when looking for comparables in vintage or antique items.

## Notes
* Given the above, we can now add to the app by
  * Instead of a keyword search over the entire internet, search eBay for specific comps
  * Get listings that seem close, then send to GPT-4 to help find the "best" comps automatically

In [59]:
def compare_item_images(base_img_url: str, comp_image_urls: list):
    """
    Compares a base image against a list of comparison images using GPT-4 with vision capabilities.

    Args:
        base_img_url (str): The URL of the base image to compare against.
        comp_image_urls (list): A list of URLs for the comparison images.

    Returns:
        The response from the API call.
    """
    # Construct the messages payload
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "I'm planning to list the item in the first image on eBay and looking for good 'comps' to help determine the price. Please review and assess whether the following images are similar enough to use. Include the number positions, if any, for the best comps, a highly concise explanation as to why.",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": base_img_url,
                    },
                }
            ],
        }
    ]

    # Dynamically add comparison images to the messages payload
    for comp_image_url in comp_image_urls:
        messages[0]["content"].append(
            {
                "type": "image_url",
                "image_url": {
                    "url": comp_image_url,
                },
            }
        )

    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=messages,
        max_tokens=300,
    )

    return response.choices[0]

In [58]:
image_urls = [str(sorted_results[i].item.image_url) for i in range(8)]
for url in image_urls:
    display(Image(url=url))

In [60]:
test_mutiple_comps = compare_item_images(target_image, image_urls)
Markdown(test_mutiple_comps.message.content)

Based on the provided images, the following are the best comps:

- Image 5: This fixture is similar as it appears to be an industrial or vintage light fixture with a protective cage and similar overall design elements. It closely matches the style and era of the item you want to list.
- Image 9: This item has a similar industrial appearance and protective cage, which might be indicative of the same or similar use and era, making it a decent comp for your item.

Images 2, 3, 4, 7 and 8 do not resemble the original item closely either in form, design, or intended use, hence they are not suitable comps. Image 6 is a duplicate of Image 5.

___

# WIP

In [ ]:
def get_llm_response(system='You are a helpful assistant.', user = '', temperature = 0.1, model = 'gpt-3.5-turbo'):
    completion = openai.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {'role': 'system', 'content': system},
            {'role': 'user', 'content': user},
        ]
    )
    return completion.choices[0].message.content

In [ ]:
def create_keyword_query_generation_prompt(topic, n):
    return f"""I'm writing a research report on {topic} and need help coming up with Google keyword search queries.
Google keyword searches should just be a few words long. It should not be a complete sentence.
Please generate a diverse list of {n} Google keyword search queries that would be useful for writing a research report on ${topic}. Do not add any formatting or numbering to the queries."""


In [ ]:
def generate_search_queries(topic, n):
    user_prompt = create_keyword_query_generation_prompt(topic, n)
    completion = get_llm_response(
        system='The user will ask you to help generate some search queries. Respond with only the suggested queries in plain text with no extra formatting, each on it\'s own line.',
        user=user_prompt,
        temperature=1
    )
    queries = [s for s in completion.split('\n') if s.strip()][:n]
    return queries

In [ ]:
prop_queries = generate_search_queries(PROPERTY_TOPIC, 3)
struct_queries = generate_search_queries(STRUCTURE_TOPIC, 3)

In [ ]:
def get_search_results(queries, type, linksPerQuery=1):
    results = []
    for query in queries:
        search_response = metaphor.search(query, type=type, num_results=linksPerQuery, use_autoprompt=False)
        results.extend(search_response.results)
    return results

In [ ]:
links = get_search_results(prop_queries, 'neural')

In [ ]:
def display_base_models(base_models):
    """
    Display the title, url, and published_date of each BaseModel object in a list.

    Args:
        base_models (List[BaseModel]): List of BaseModel objects.

    """
    for model in base_models:
        print(f"Title: {model.title}")
        print(f"URL: {model.url}")
        print(f"Published Date: {model.published_date}")

In [ ]:
display_base_models(links)

In [ ]:
from bs4 import BeautifulSoup
import re

def clean_html_content(content: str) -> str:
    """
    Clean the HTML content using BeautifulSoup.

    Args:
        content (str): HTML content.

    Returns:
        str: Cleaned text content.
    """
    soup = BeautifulSoup(content, "html.parser")

    # Extract header and paragraph tags
    header_tags = soup.find_all(re.compile(r"^h\d$"))
    paragraph_tags = soup.find_all("p")

    # Strip HTML tags and collect text content
    stripped_content = ""
    for tag in header_tags + paragraph_tags:
        stripped_content += " " + tag.get_text().strip() + " "

    return ' '.join(stripped_content.split())

In [ ]:
def get_page_contents(search_results):
    contents_response = metaphor.get_contents(search_results)
    return contents_response.contents


content = get_page_contents([link.id for link in links])

In [ ]:
def create_web_content_string(search_contents: list, char_limit: int = 30000) -> str:
    """
    Synthesize a report from search contents.

    Args:
        search_contents (list): List of search contents.
        char_limit (int, optional): Total character limit. Defaults to 30000.

    Returns:
        str: Synthesized report.
    """
    total_chars = sum([len(clean_html_content(item.extract)) for item in search_contents])
    inputData = ''

    for item in search_contents:
        cleaned_content = clean_html_content(item.extract)
        item_chars = len(cleaned_content)
        slice_ratio = item_chars / total_chars
        slice_limit = int(char_limit * slice_ratio)
        sliced_content = cleaned_content[:slice_limit]

        inputData += f'--START ITEM--\nURL: {item.url}\nTITLE: {item.title}\nCONTENT: {sliced_content}\n--END ITEM--\n'

    return inputData
    # return get_llm_response(
    #     system='You are a helpful research assistant. Write a report according to the user\'s instructions.',
    #     user='Input Data:\n' + inputData + f'Write a two paragraph research report about {topic} based on the provided information. Include as many sources as possible. Provide citations in the text using footnote notation ([#]). First provide the report, followed by a single "References" section that lists all the URLs used, in the format [#] .',
    #     model='gpt-4' # want a better report? use gpt-4
    # )

In [ ]:
def format_for_markdown(text: str) -> str:
    """
    Formats the given text for markdown.

    Args:
        text (str): The text to be formatted.

    Returns:
        str: The formatted text.
    """
    # Split the text into items
    items = text.split("--END ITEM--")
    
    # Process each item
    formatted_items = []
    for item in items:
        if item.strip() == "":
            continue

        # Remove START ITEM tag and split into lines
        lines = item.replace("--START ITEM--", "").strip().split(" ")

        # Initialize formatted item
        formatted_item = "\n\n"

        # Add each line with a newline at the end
        for line in lines:
            if "URL:" in line or "TITLE:" in line:
                formatted_item += "<br/>" + line
            elif "CONTENT:" in line:
                formatted_item += "<br/>" + line + "<br/>"
            else:
                formatted_item += " " + line

        formatted_items.append(formatted_item.strip())

    return "<br/><br/>".join(formatted_items)

# Your provided text
text = """
--START ITEM-- URL: https://www.stanley12volt.com/ TITLE: HOME | stanley12volt CONTENT: WATCH & LEARN Watch and Learn Learn more about your favorite products! Check out the Watch & Learn Section to see more videos Subscribe for Updates Congrats! You’re subscribed --END ITEM-- --START ITEM-- URL: https://www.woodcraft.com/categories/planes TITLE: Hand Planes for Sale from WoodRiver, Stanley, Veritas & More | Woodcraft CONTENT: Woodworking Hand Planes This Simple Yet Versatile Tool Deserves a Place In Your Workshop Hand Plane Resources & Videos Hand Planes are deceptively simple tools that when used properly can deliver speed and precision not possible with power tools or abrasives. Prior to the introduction of motorized power tools, hand planes bridged the gap between rough lumber and finished projects. A properly tuned hand plane can give you superior results when it comes to smoothing and other fine work. Learn more about this versatile tool through the articles and videos below, and be sure to check out our Woodcraft's Hand Plane Buying Guide! Top of Page --END ITEM-- --START ITEM-- URL: https://www.k-bid.com/auction/44829/item/203 TITLE: Vintage Stanley Woodworking Planer No. 602 CONTENT: Pick up is located at the back entrance of the warehouse.
"""

# Format the text for markdown
formatted_text = format_for_markdown(text)
print(f"{formatted_text}")

In [ ]:
def clean_html_content(content: str) -> str:
    """
    Clean the HTML content using BeautifulSoup.

    Args:
        content (str): HTML content.

    Returns:
        str: Cleaned text content.
    """
    soup = BeautifulSoup(content, "html.parser")

    # Extract header and paragraph tags
    header_tags = soup.find_all(re.compile(r"^h\d$"))
    paragraph_tags = soup.find_all("p")

    # Strip HTML tags and collect text content
    stripped_content = ""
    for tag in header_tags + paragraph_tags:
        stripped_content += " " + tag.get_text().strip() + " "

    return " ".join(stripped_content.split())


def create_web_content_string(search_contents: list, char_limit: int = 9000) -> str:
    """
    Build context for LLM call.

    Args:
        search_contents (list): List of search contents.
        char_limit (int, optional): Total character limit. Defaults to 9000.

    Returns:
        str: Processed internet content.
    """
    total_chars = sum(
        [len(clean_html_content(item.extract)) for item in search_contents]
    )
    internet_content = ""

    for item in search_contents:
        cleaned_content = clean_html_content(item.extract)
        item_chars = len(cleaned_content)
        slice_ratio = item_chars / total_chars
        slice_limit = int(char_limit * slice_ratio)
        sliced_content = cleaned_content[:slice_limit]

        internet_content += f"--START ITEM--\nURL: {item.url}\nTITLE: {item.title}\nCONTENT: {sliced_content}\n--END ITEM--\n"

    return internet_content


def synthesize_report(topic: str, internet_content: str) -> str:
    return openai.chat.completions.create(
        model="gpt-4-1106-preview",
        temperature=1,
        messages=[
            {
                "role": "system",
                "content": "You are a helpful internet research assistant specializing in empowering buyers. You help sift through raw search results to find the most relevant and interesting findings for user topic of interest.",
            },
            {
                "role": "user",
                "content": "Input Data:\n"
                + internet_content
                + f"Write a two paragraph research report about **{topic}** based on the provided search results. One paragraph summarizing the Input Data, and another focusing on the main Research Topic. Include as many sources as possible. ALWAYS cite results using [[number](URL)] notation after the reference. End with a markdown table of all the URLs used. Remember to use markdown links when citing the context, for example [[number](URL)].",
            },
        ],
        )
    


In [ ]:
response = metaphor.search("ceiling drywall",
    num_results=5,
    type='keyword',
    start_published_date="2023-06-12"
)

print(response)

In [ ]:
def researcher(topic, n_queries, n_links_per_query):
    search_queries = generate_search_queries(topic, n_queries)
    print(search_queries)
    search_results = get_search_results(search_queries, 'neural', n_links_per_query)
    search_contents = get_page_contents([link.id for link in search_results])
    internet_content = create_web_content_string(search_contents, 30000)
    report = synthesize_report(topic, internet_content)
    return report

In [ ]:
research = researcher(PROPERTY_TOPIC, 2, 2)

Markdown(f"{research.choices[0].message.content}")

In [ ]:
from langchain.tools import tool
from crewai import Agent, Task, Crew, Process

@tool
def duckduckgo_search_tool(query: str) -> str:
    """
    Tool to perform DuckDuckGo searches and return formatted results.
    The input should be a string representing the search query.
    This function has been updated to return a single string that formats all values from each dict in the list of results.

    Args:
        query (str): The search query.

    Returns:
        str: A formatted string containing titles, snippets, and links from all search results.
    """
    # Initialize the DuckDuckGo Search API wrapper
    duckduckgo_serper = DuckDuckGoSearchAPIWrapper()
    # Run the search
    results = duckduckgo_serper.results(query, max_results=4)
    # Format each result into a readable string
    formatted_results = []
    for result in results:
        formatted_result = f"Title: {result['title']}\nSnippet: {result['snippet']}\nLink: {result['link']}\n---"
        formatted_results.append(formatted_result)
    # Join all formatted results into a single string
    return "\n\n".join(formatted_results)

# Initialize the Research Analyst agent and assign the custom tool
research_analyst = Agent(
    role='Research Analyst',
    goal='Find comparable listings for items using DuckDuckGo Search',
    backstory='Skilled in web scraping and market research, using DuckDuckGo searches to gather data on similar items for sale.',
    verbose=True,
    tools=[duckduckgo_search_tool]
)

# Task for the Research Analyst: Scrape comparable listings
scrape_comps_task = Task(
    description='Use DuckDuckGo Search to find and scrape comparable listings for a provided list of items.',
    agent=research_analyst,
    final_answer='Return a structured dataset with comparable listings data.',
    expected_output='A JSON or structured format containing the titles, links, and prices of comparable listings.' 
)


# Define the Data Analyst agent
data_analyst = Agent(
    role='Data Analyst',
    goal='Analyze the data from comparable listings to provide pricing recommendations',
    backstory='Expert in data analysis and pricing strategy, transforming data into actionable insights.',
    verbose=True
)

# Task for the Data Analyst: Build pricing recommendations
build_pricing_recommendations_task = Task(
    description='Analyze the scraped data to determine competitive pricing recommendations for each item.',
    agent=data_analyst,
    final_answer='Provide a list of pricing recommendations for each item based on the analysis.',
    expected_output='A list of items with their recommended pricing based on the market analysis.' 
)

# Assemble the crew
crew = Crew(
    agents=[research_analyst, data_analyst],
    tasks=[scrape_comps_task, build_pricing_recommendations_task],
    process=Process.sequential,
    verbose=True
)

In [ ]:
result = crew.kickoff()
print(result)